In [1]:
import numpy as np 
import pandas as pd
import torch
from sklearn.metrics import r2_score
from pyro import clear_param_store
import pyro as pyro
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import arviz as az
from scipy.optimize import curve_fit


# Importing data 

In [2]:
PM25=pd.read_excel("../data/Pre_flow_cal.xlsx")
train,test=PM25.loc[PM25.label==0,:],PM25.loc[PM25.label==1,:]

In [3]:
extra_data_end=train.loc[train.corrected_week<20,:]
extra_data_start=train.loc[train.corrected_week>52.3-20,:]
extra_data_end.corrected_week=extra_data_end.corrected_week+52.3
extra_data_start.corrected_week=-(52.3-extra_data_start.corrected_week)
extra_data_start=extra_data_start.set_index(np.zeros(extra_data_start.shape[0]))
extra_data_end=extra_data_end.set_index(np.zeros(extra_data_end.shape[0]))
train=pd.concat([train,extra_data_end,extra_data_start])

/tmp/ipykernel_2831468/2357358645.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_end.corrected_week=extra_data_end.corrected_week+52.3
/tmp/ipykernel_2831468/2357358645.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_start.corrected_week=-(52.3-extra_data_start.corrected_week)


# Converting  PM25 data to tensors and put on GPU

In [4]:
# put data on cuda else cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#make train and test data ready for flow of pump vs pm2.5 ready 
X25=torch.tensor(train.corrected_week.values).float().to(device)
y25=torch.tensor(train.flow.values).float().to(device)
X25_test=torch.tensor(test.flow.values).float().to(device)
y25_test=test.pm_final.values

In [5]:
clear_param_store();
rbf = gp.kernels.RBF(input_dim=1);

rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(PM25.flow.mean())))

rbf.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(15.)))
gpr = gp.models.GPRegression(X25,y25, rbf).to(device);
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(2.)))

nuts_kernel = NUTS(gpr.model);

mcmc = MCMC(nuts_kernel, num_samples=1000,warmup_steps=9000,num_chains=1);

mcmc.run();

Sample: 100%|██████████████████████████████████████| 10000/10000 [03:18, 50.26it/s, step size=7.37e-01, acc. prob=0.896]


In [6]:
torch.save(gpr, "../models/flow_season");

In [7]:
posterior_samples = mcmc.get_samples();
posterior_predictive = Predictive(gpr, posterior_samples)(X25);
prior = Predictive(gpr, num_samples=500)(X25);

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive

);
az.to_json(pyro_data, "../Arviz_stats/mcmc_season_flow.json")

/data/michaelf/miniconda3/envs/gaussion_process/lib/python3.11/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_season_flow.json'

In [8]:
az.style.use("arviz-doc")

ax = az.plot_trace(pyro_data,backend='bokeh')

In [22]:
0.

0.0